In [40]:

%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:

import numpy as np
import matplotlib.pyplot as plt
from fgbuster import CMB, Dust, Synchrotron, MixingMatrix
from fgbuster.observation_helpers import standardize_instrument, get_observation
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import healpy as hp
import numdifftools as nd
import math as m

In [42]:
instr = np.load('/Users/alicepirotta/Desktop/APC/MCMC/instrument_LB_IMOv1.npy', allow_pickle=True).item()
instr_ = {}
instr_['frequency'] = np.array([instr[f]['freq'] for f in instr.keys()])
instr_['depth_p'] = np.array([instr[f]['P_sens'] for f in instr.keys()])
instr_['fwhm'] = np.array([instr[f]['beam'] for f in instr.keys()])
instr_['depth_i'] = instr_['depth_p']/np.sqrt(2)
instrument = standardize_instrument(instr_)


In [43]:
nside = 4
freq_maps = get_observation(instrument, 'd0s0', noise=True, nside=nside)


In [44]:

components= [CMB(), Dust(50.), Synchrotron(50.)]

In [45]:

A = MixingMatrix(*components)
A_ev = A.evaluator(instrument.frequency)
#invN=np.linalg.inv(np.eye(len(instrument.frequency)))
invN = np.diag((hp.nside2resol(nside, arcmin=True) / instrument.depth_p)**2)

LIKELIHOOD WITHOUT GAIN

In [46]:
def spectral_likelihood(x):
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =A_ev(x) 
    print(A_maxL.shape)
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(x,logL)
    if logL != logL:
        return 0.0
    return -logL

In [47]:
np.random.seed(10)
x0 =np.array([1.54,20,-3])
pos = np.random.uniform(low=x0 * (1 - 1 / 4), high= x0 * (1 + 1 / 4))
#min_= minimize (spectral_likelihood,x0,method='TNC')
min_= minimize (spectral_likelihood,pos,method='TNC',options={'maxfun':1000000000},tol= 1e-18 ) 
print(min_)
print(min_.x)

(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)


In [48]:
print(min_.x)

[ 1.6190291  17.5288192  -3.52535534]


In [49]:
# x0= np.array([1.54,20,-3])
# invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
# A_maxL =A_ev(min_.x) 
# AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
# AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
# s = AtNA.dot(AtNd)
# ps_CMB =hp.anafast(s[0])
# #ps_CMB =hp.sphtfunc.anafast(s[0])
# plt.loglog(ps_CMB)
# print (ps_CMB.shape)
# plt.xlabel(r'$\ell$')



In [50]:
# x0= np.array([1.54,20,-3])
# invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
# A_maxL =A_ev(min_.x) 
# AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
# AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
# s = AtNA.dot(AtNd)

# ps_CMB =hp.anafast(s[0])
# ps_CMB=ps_CMB.T
# ps_CMB=ps_CMB[2:,:]

# ell = np.arange(2,192)

# values = []
# for i in range(len(ps_CMB[0,:])):
#     y = (ell*(ell+1)*ps_CMB[:,i])/(2*m.pi)
#     values.append(y)


# plt.loglog(ell,values[0], label="TT")
# plt.loglog(ell,values[1], label="EE")
# plt.loglog(ell,values[2], label="BB")
# plt.loglog(ell,values[3], label="TE")
# plt.loglog(ell,values[4], label="EB")
# plt.loglog(ell,values[5], label="TB")


# plt.xlabel(r'$\ell$')
# plt.ylabel(r'$\ell$* ($\ell$+1)*C$\ell$/2$\pi$')
# plt.legend()

In [51]:

print(spectral_likelihood(x0))

(22, 3)
-31595865045584.223


hessian matrix

In [52]:
H = nd.Hessian(spectral_likelihood)
h = H([1.54, 20, -3])
print(h)




(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)


eigen vector, eigen values

In [53]:
# w, v = np.linalg.eig(h)
# print(w,v)
# plt.plot(v)

LIKELIHOOD WITH GAIN (1 variables, 1 fixed)

In [54]:
def spectral_likelihood(y):
    Bd, T, Bs, a = y
    b = 1
    G = np.diag([a,a,a,a,a,a,a,a,a,a,a,b,b,b,b,b,b,b,b,b,b,b])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    print(A_maxL.shape)
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [55]:
# x0= np.array([1.54,20,-3,1])
# a, b = 1, 1
# G = np.diag([a,a,a,a,a,a,a,a,a,a,a,b,b,b,b,b,b,b,b,b,b,b])
# invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
# A_maxL =G.dot(A_ev(x0)) 
# AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
# AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
# s = AtNA.dot(AtNd)
# Q =hp.sphtfunc.anafast(s[0])
# plt.plot(Q)


In [56]:
x0 =np.array([1.4, 21, -2.5, 1])  
x1 = np.random.uniform(low=[x0[0] * (1 - 1 / 40), x0[1] * (1 - 1 / 40), x0[2] * (1 - 1 / 40),  x0[3] * (1 - 1 / 40)], high=[x0[0] * (1 + 1 / 40), x0[1] * (1 + 1 / 40), x0[2] * (1 + 1 / 40),  x0[3] * (1 + 1 / 40)], size=(len(x0)))

min_= minimize (spectral_likelihood,x1,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)

(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)
(22, 3)


In [57]:
print(min_.x)

[ 1.53004865 20.33788987 -2.74864695  0.99985417]


LIKELIHOOD WITH GAIN (2 variables, 1 fixed)

In [58]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c = y
    b = 1
    G = np.diag([a,a,a,a,a,a,a,b,b,b,b,b,b,b,b,c,c,c,c,c,c,c])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [59]:
x0 =np.array([1.4, 21, -2.5, 1, 1])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)

          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595864217407.902
                x: [ 1.519e+00  2.073e+01 -2.750e+00  9.986e-01  9.991e-01]
              nit: 224
             nfev: 1308
             njev: 218
             nhev: 0
         cg_niter: 958
     cg_stop_cond: 2
             grad: [ 4.658e+06  2.630e+06 -5.528e+06  8.913e+06  9.437e+06]
  lagrangian_grad: [ 4.658e+06  2.630e+06 -5.528e+06  8.913e+06  9.437e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 9437184.0
 constr_violation: 0
   execution_time: 0.6870789527893066
        tr_radius: 6.48185868187902e-19
   constr_penalty: 1.0
            niter: 224
[ 1.51948601 20.73089502 -2.75025715  0.99858334  0.99911084]


In [60]:
H = nd.Hessian(spectral_likelihood)
h = H([1.54, 20, -3, 1.1, 1.5])
print(h)

[[ 7.57719732e+11  2.14803557e+10  1.46417586e+10  1.04199175e+09
  -1.85703290e+11]
 [ 2.14803557e+10  1.01105647e+09  3.46420702e+08 -2.97635659e+07
  -4.73197475e+09]
 [ 1.46417586e+10  3.46420702e+08 -9.39467005e+08  4.88844911e+09
  -4.88723331e+09]
 [ 1.04199175e+09 -2.97635659e+07  4.88844911e+09  4.83593447e+09
   1.22488775e+10]
 [-1.85703290e+11 -4.73197475e+09 -4.88723331e+09  1.22488775e+10
   1.13340181e+11]]


In [61]:
# w, v = np.linalg.eig(h)
# print(w,v)
# plt.plot(v.T)

LIKELIHOOD WITH GAIN ( 5 variables, 1 fixed)

In [62]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c, d, e, f = y
    b = 1
    G = np.diag([a,a,a,a,b,b,b,b,c,c,c,c,d,d,d,d,e,e,e,f,f,f])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [63]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595864861038.54
                x: [ 1.533e+00  2.024e+01 -2.827e+00  9.969e-01  1.001e+00
                     1.001e+00  1.001e+00  1.001e+00]
              nit: 726
             nfev: 6480
             njev: 720
             nhev: 0
         cg_niter: 5058
     cg_stop_cond: 2
             grad: [ 6.670e+06  5.181e+05 -2.781e+05  3.932e+06  4.188e+06
                     1.257e+07  1.440e+07  5.760e+06]
  lagrangian_grad: [ 6.670e+06  5.181e+05 -2.781e+05  3.932e+06  4.188e+06
                     1.257e+07  1.440e+07  5.760e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 14400718.24292973
 constr_violation: 0
   execution_time: 3.1331520080566406
        tr_radius: 9.039021838185776e-19
   constr_penalty: 

LIKELIHOOD WITH GAIN (8 variables,1 fixed)

In [64]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c, d, e, f, g, h, i  = y
    b = 1
    G = np.diag([a, a, a, b, b, c, c, c, d, d, e, e, f, f, f, g, g, h, h, i, i, i ])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [65]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.5, 1.5, 0.8])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595864382296.57
                x: [ 1.515e+00  2.091e+01 -2.803e+00  9.963e-01  1.005e+00
                     1.005e+00  1.004e+00  1.004e+00  1.004e+00  1.004e+00
                     1.005e+00]
              nit: 504
             nfev: 5976
             njev: 498
             nhev: 0
         cg_niter: 4001
     cg_stop_cond: 2
             grad: [ 1.021e+07  1.767e+06 -7.388e+06  1.127e+07  1.748e+07
                     1.487e+07  1.175e+07  2.114e+07  1.201e+07  1.749e+07
                     1.356e+07]
  lagrangian_grad: [ 1.021e+07  1.767e+06 -7.388e+06  1.127e+07  1.748e+07
                     1.487e+07  1.175e+07  2.114e+07  1.201e+07  1.749e+07
                     1.356e+07]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_s

LIKELIHOOD WITH GAIN (11 variables, 1 fixed)

In [66]:
def spectral_likelihood(y):
    Bd, T, Bs, a, b, c, d, e, g, h, i, l, m, n  = y
    f = 1
    G = np.diag([a, a, b, b, c, c, d, d, e, e, g, g, h, h, i, i, l, l, m, m, n, n ])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [67]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.5, 1.5, 0.8, 1.2,0.6,0.3])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: The maximum number of function evaluations is exceeded.
          success: False
           status: 0
              fun: -31595864068830.4
                x: [ 1.513e+00  2.092e+01 ...  1.096e+00  1.097e+00]
              nit: 1000
             nfev: 15000
             njev: 1000
             nhev: 0
         cg_niter: 11371
     cg_stop_cond: 2
             grad: [ 5.025e+06  5.513e+05 ... -1.004e+07  5.258e+06]
  lagrangian_grad: [ 5.025e+06  5.513e+05 ... -1.004e+07  5.258e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 25822984.52584314
 constr_violation: 0
   execution_time: 6.520982027053833
        tr_radius: 7.386120279250995e-05
   constr_penalty: 1.0
            niter: 1000
[ 1.51296765 20.92194318 -2.63143151  1.07837638  1.08634186  1.0933106
  1.09571703  1.0972934   1.09691728  1.09727774  1.09637025  1.

LIKELIHOOD WITH GAIN (21 variables, 1 fixed)

In [68]:
def spectral_likelihood(y):
    Bd, T, Bs, a, b, c, d, e, f, g, h, i, l, m, n, o, p, q, r, t, u, v, w, z = y
    s = 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [69]:
#np.random.seed(10)
x0 =np.array([1.54,20,-3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
pos = np.random.uniform(low=x0 * (1 - 1 / 4), high= x0 * (1 + 1 / 4))
min_= minimize (spectral_likelihood,pos,method='TNC',options={'maxfun':1000000000},tol= 1e-18 ) 
print(min_)
print(min_.x)


 message: Converged (|f_n-f_(n-1)| ~= 0)
 success: True
  status: 1
     fun: -31594993617657.523
       x: [ 1.588e+00  1.648e+01 ...  1.032e+00  1.044e+00]
     nit: 41
     jac: [-3.285e+08 -3.750e+07 ...  2.306e+10 -1.718e+10]
    nfev: 7175
[ 1.58750186 16.47564145 -2.37741082  1.09004035  1.22607305  0.75728497
  1.00583174  1.1493715   1.04651958  1.10501283  0.90084     1.17744955
  1.08020632  1.00002523  1.01729515  0.94215618  1.01034929  1.01199507
  1.01008206  1.0124214   1.01497292  1.02035004  1.03151806  1.04362443]


In [70]:

Fisher = nd.Hessian(spectral_likelihood)
F = Fisher(min_.x)
print(F)



[[ 8.51990101e+11  3.60151747e+10  2.51333590e+09 -6.04152487e+09
  -3.86958657e+09 -9.27428110e+08  8.14843229e+08  4.11926985e+08
   6.49890648e+09  4.75765935e+09  1.27654451e+10  3.94299143e+09
   3.39709870e+10  6.22846642e+10  5.44487561e+10  3.84233309e+10
   1.15723286e+11  6.91767026e+10 -7.70715650e+10 -1.63621051e+11
  -6.28657293e+11 -1.13279819e+12 -6.57038461e+11  2.56344519e+12]
 [ 3.60151747e+10  1.54501729e+09  9.45234758e+07 -2.22000495e+08
  -1.48575125e+08 -4.11891005e+07  1.35279799e+07  6.83879468e+06
   2.14489532e+08  1.57021511e+08  4.53464272e+08  1.40066066e+08
   1.25658309e+09  2.45216548e+09  2.41981079e+09  1.42127482e+09
   4.55605967e+09  3.07434992e+09 -1.87115708e+09 -6.03378668e+09
  -2.56688961e+10 -5.05021279e+10 -3.67363839e+10  1.15661761e+11]
 [ 2.51333590e+09  9.45234758e+07  3.56653288e+07 -1.01898084e+08
  -2.09320218e+07  1.82199937e+07  6.71851829e+07  3.39514368e+07
   1.38001139e+08  1.01013036e+08  1.57165770e+08  4.85189102e+07
   2.717

In [71]:
Nbd=64 
sqr_Nbd=np.sqrt(Nbd)
Ntd=8
sqr_Ntd= np.sqrt(Ntd)
Nbs=4
sqr_Nbs=np.sqrt(Nbs)
g=1
Npatch= np.matrix([[Nbd,np.sqrt(Nbd*Ntd),np.sqrt(Nbd*Nbs),sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd,sqr_Nbd], [np.sqrt(Ntd*Nbd),Ntd,np.sqrt(Ntd*Nbs),sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd,sqr_Ntd], 
                [np.sqrt(Nbs*Nbd),np.sqrt(Nbs*Ntd),Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Ntd,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs,sqr_Nbs], [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],
                [sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g],[sqr_Nbd,sqr_Ntd,sqr_Nbs,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g] ])

In [72]:
# print(Npatch)

In [73]:
# F1= F*Npatch

# print(F1)


In [74]:

# sqrt_inv_F1= np.sqrt(np.linalg.inv(F1))
# print(sqrt_inv_F1)


In [75]:

# sqrt_inv_F= np.sqrt(np.linalg.inv(F))
# print(sqrt_inv_F)

In [76]:
# w, v = np.linalg.eig(h)
# print(w,v)
# plt.plot(v)

LIKELIHOOD WITH GAIN (11 variables, the low frequency band varies)

In [77]:
def spectral_likelihood(y):
    Bd, T, Bs, a, b, c, d, e, f, g, h, i, l, m, = y
    n, o, p, q, r, s, t, u, v, w, z = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [78]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.6, 0.3, 1.5, 0.5, 1.5, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595863377035.473
                x: [ 1.513e+00  2.100e+01 ...  1.001e+00  1.001e+00]
              nit: 254
             nfev: 3720
             njev: 248
             nhev: 0
         cg_niter: 1568
     cg_stop_cond: 2
             grad: [ 1.750e+07  2.959e+06 ...  1.362e+07  5.764e+06]
  lagrangian_grad: [ 1.750e+07  2.959e+06 ...  1.362e+07  5.764e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 17820352.332335114
 constr_violation: 0
   execution_time: 1.6319496631622314
        tr_radius: 6.248177513071598e-19
   constr_penalty: 1.0
            niter: 254
[ 1.51266111 20.99841767 -2.49836417  1.05925304  1.01079551  0.99924449
  0.99837363  0.99872078  0.99950199  0.99935169  1.00030525  1.00036793
  1.0

LIKELIHOOD WITH GAIN ( 11 variables, every two gain)

In [79]:
def spectral_likelihood(y):
    Bd, T, Bs, a, c, e, g, i, m, o, q, s, u, w = y
    b, d, f, h, l, n, p, r, t, v, z = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [80]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.6, 0.3, 1.5, 0.5, 1.5, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: The maximum number of function evaluations is exceeded.
          success: False
           status: 0
              fun: -31595864132653.566
                x: [ 1.513e+00  2.099e+01 ...  9.998e-01  1.000e+00]
              nit: 1000
             nfev: 15000
             njev: 1000
             nhev: 0
         cg_niter: 9640
     cg_stop_cond: 2
             grad: [-8.144e+06  7.492e+05 ... -4.981e+06  2.621e+05]
  lagrangian_grad: [-8.144e+06  7.492e+05 ... -4.981e+06  2.621e+05]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 15728640.0
 constr_violation: 0
   execution_time: 6.217097997665405
        tr_radius: 5.586816698779412e-08
   constr_penalty: 1.0
            niter: 1000
[ 1.51283963 20.9930987  -2.52032558  1.06389201  1.00211904  1.00013426
  1.00004371  1.00044973  1.00045925  1.00062729  1.00007752  0.99965

LIKELIHOOD WITH GAIN ( 11 variables, the middle and the high frequency band varies )

In [81]:
def spectral_likelihood(y):
    Bd, T, Bs, n, o, p, q, r, s, t, u, v, w, z = y
    a, b, c, d, e, f, g, h, i, l, m = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [82]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2, 0.6, 0.3, 1.5, 0.5, 1.5, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', tol= 1e-18)
print(min_)
print(min_.x)


          message: The maximum number of function evaluations is exceeded.
          success: False
           status: 0
              fun: -31595863084089.223
                x: [ 1.209e+00  2.099e+01 ...  1.207e+00  1.268e+00]
              nit: 1000
             nfev: 15000
             njev: 1000
             nhev: 0
         cg_niter: 11794
     cg_stop_cond: 1
             grad: [-4.338e+05  4.371e+05 ... -2.303e+07  2.585e+07]
  lagrangian_grad: [-4.338e+05  4.371e+05 ... -2.303e+07  2.585e+07]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
           method: equality_constrained_sqp
       optimality: 25851042.899886895
 constr_violation: 0
   execution_time: 6.794980049133301
        tr_radius: 0.0001662096214881066
   constr_penalty: 1.0
            niter: 1000
[ 1.20854382 20.99212167 -2.64502595  1.01424841  0.99590152  1.00146608
  1.01423782  1.03542957  1.0627143   1.06269693  1.10307654

LIKELIHOOD WITH GAIN ( 5 variables, the high frequency band varies)

In [83]:
def spectral_likelihood(y):
    Bd, T, Bs, t, u, v, w, z  = y
    a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s = 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    G = np.diag([a,b,c,d,e,f,g,h,i,l,m,n,o,p,q,r,s,t,u,v,w,z])
    invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
    A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    #print(i,logL)
    if logL != logL:
        return 0.0
    return -logL

In [84]:
x0 =np.array([1.4, 21, -2.5, 1.1, 0.4, 1.5, 0.8, 1.2])  
min_= minimize (spectral_likelihood,x0,method='trust-constr', options={'maxiter': 1000000, 'disp': True} , tol= 1e-18)
print(min_)
print(min_.x)


`xtol` termination condition is satisfied.
Number of iterations: 279, function evaluations: 2448, CG iterations: 1656, optimality: 1.03e+07, constraint violation: 0.00e+00, execution time:  1.6 s.
          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: -31595864996525.01
                x: [ 1.521e+00  2.098e+01 -2.899e+00  1.000e+00  1.000e+00
                     9.997e-01  9.989e-01  9.975e-01]
              nit: 279
             nfev: 2448
             njev: 272
             nhev: 0
         cg_niter: 1656
     cg_stop_cond: 2
             grad: [ 1.034e+07  4.123e+05 -2.170e+06  4.456e+06  6.029e+06
                     5.243e+06  5.505e+06  4.719e+06]
  lagrangian_grad: [ 1.034e+07  4.123e+05 -2.170e+06  4.456e+06  6.029e+06
                     5.243e+06  5.505e+06  4.719e+06]
           constr: []
              jac: []
      constr_nfev: []
      constr_njev: []
      constr_nhev: []
                v: []
   

In [85]:
# def spectral_likelihood(i):
#     Bd, T, Bs, a, b, c, d, e, f, g, h, i, l, m, n, o, p, q, r, s, t, u, v, w, z = i
#     h = 1
#     G = np.diag([ a, b, c, d, e, f, g, i, h, l, m, n, o, p, q, r, s, t, u, v, w, z])
#     invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
#     A_maxL =G.dot(A_ev(np.array([Bd,T,Bs]))) 
#     logL = 0
#     AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
#     AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
#     logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
#     #print(i,logL)
#     if logL != logL:
#         return 0.0
#     return -logL

In [86]:
# tol_ = 1e-18
# x0 =np.array([1.3, 15, -2.5, 0.5, 1.2, 0.1, 2, 0.2, 0.9, 1.1, 0.3, 0.9, 1.4, 1.2, 0.4, 0.8, 0.9, 1.2, 1.3, 0.7, 0.2, 1.9, 0.9, 1.3, 1.4]) 
# min_= minimize (spectral_likelihood,x0,method='trust-constr')
# print(min_)
# print(min_.x)


In [87]:
# g1 = np.linspace(0.5,1.5,500)
# result= []
# for i in g1:
#         result.append(spectral_likelihood(np.array([1.26466502, 17.99580668 , -2.65181336 , 1.02481412 , 1.13362434 , 1.13395641,
#   1.10611243 , 1.10360917 ,i  , 1.06613316 , 1.03197411 , 1.03194025,
#   1.00891511,  0.98736361 , 0.98027652,  1.00900499 , 0.98735861  ,0.98027463,
#   0.9850154  , 1.00000578,  1.02959818 , 1.06909778 , 1.12300709 , 1.18561135])))
# print(result)

# plt.plot(g1,result,'o')
# plt.title(r'Likelihood varying g1')
# plt.xlabel(r'slope index, g1')
# plt.ylabel('Likelihood value')

In [88]:
# Bd = np.linspace(1.4,1.6,20)
# g3 = np.linspace(0.8,1.2,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(g3):
#         result[i,j] = spectral_likelihood(np.array([val_i,20,-3,1,1,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, g3, result.T,100)
# ax.contour(Bd, g3, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'g3')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [89]:
# T = np.linspace(15,25,20)
# g3 = np.linspace(0.5,1.5,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(T):
#     for j,val_j in enumerate(g3):
#         result[i,j] = spectral_likelihood(np.array([1.54,val_i,-3,1,1,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(T, g3, result.T,100)
# ax.contour(T, g3, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'T')
# ax.set_ylabel(r'g3')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [90]:
# Bs = np.linspace(-3.2,-2.8,20)
# g3 = np.linspace(0.8,1.2,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bs):
#     for j,val_j in enumerate(g3):
#         result[i,j] = spectral_likelihood(np.array([1.54,20,val_i,1,1,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bs, g3, result.T,100)
# ax.contour(Bs, g3, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_s$')
# ax.set_ylabel(r'g3')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
# plt.show()

In [91]:
# Bd = np.linspace(1.0,2.5,20)
# Bs = np.linspace(-4,-2,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(Bs):
#         result[i,j] = spectral_likelihood(np.array([val_i,20,val_j]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, Bs, result.T,100)
# ax.contour(Bd, Bs, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'$\beta_s$')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()


In [92]:
# Bd = np.linspace(2.6,3,20)
# Bs = np.linspace(-0.5,0.5,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(Bs):
#         result[i,j] = spectral_likelihood(np.array([val_i,20,val_j]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, Bs, result.T,100)
# ax.contour(Bd, Bs, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'$\beta_s$')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [93]:
# Bd = np.linspace(1.52,1.56,20)
# T = np.linspace(19.5,20.5,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(Bd):
#     for j,val_j in enumerate(T):
#         result[i,j] = spectral_likelihood(np.array([val_i,val_j,-3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(Bd, T, result.T,100)
# ax.contour(Bd, T, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'$\beta_d$')
# ax.set_ylabel(r'T')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()

In [94]:
# T = np.linspace(19.9,20.1,20)
# Bs = np.linspace(-3.1,-2.9,20)
# result = np.zeros((20,20))
# for i,val_i in enumerate(T):
#     for j,val_j in enumerate(Bs):
#         result[i,j] = spectral_likelihood(np.array([1.54,val_i,val_j,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]))      
   
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.contourf(T, Bs, result.T,100)
# ax.contour(T, Bs, result.T,levels=[ np.max(result)-2.3])

# ax.set_xlabel(r'T')
# ax.set_ylabel(r'Betas')
# ax.ticklabel_format(style='sci', axis='both', scilimits=(0,0))

# plt.show()